In [3]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Cargar el archivo de texto en un DataFrame
df = pd.read_csv('sell-in.txt.gz', sep='\t')


In [4]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [13]:
df.columns

Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'mes', 'año', 'trimestre',
       'cust_request_tn_lag1', 'cust_request_tn_lag2', 'cust_request_tn_lag3'],
      dtype='object')

In [5]:
productos = pd.read_csv('tb_productos.txt', sep='\t')

In [6]:
productos.head()

,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288


In [7]:
df_merged = pd.merge(df, productos, on='product_id', how='left')

In [8]:
stocks = pd.read_csv('tb_stocks.txt.gz', sep='\t')

In [9]:
stocks.head()

,periodo,product_id,stock_final
0,201810,20524,1.61267
1,201810,20311,2.93657
2,201810,20654,6.83269
3,201810,21005,1.01338
4,201810,20974,0.34595


In [10]:
df_final = pd.merge(df_merged, stocks, on=['periodo', 'product_id'], how='left')

In [11]:
df_final.tail()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
2988645,201912,10105,20853,0,1,0.02230,0.02230,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,1.82373
2988646,201912,10092,20853,0,1,0.00669,0.00669,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,1.82373
2988647,201912,10006,20853,0,7,0.02898,0.02898,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,1.82373
2988648,201912,10018,20853,0,4,0.01561,0.01561,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,1.82373
2988649,201912,10020,20853,0,2,0.01561,0.01561,PC,CABELLO,Shampoo Bebe,NIVEA,200.0,1.82373


In [18]:
#df_final.to_csv('datos_unidos.csv', index=False)

In [12]:
#datosunidos = pd.read_csv('datos_unidos.zip')

In [13]:
#datosunidos.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [15]:
from autoviz.AutoViz_Class import AutoViz_Class


# Crear una instancia de AutoViz
AV = AutoViz_Class()

# Realizar EDA
df_eda = AV.AutoViz(df_final, sep='\t')




    Since nrows is smaller than dataset, loading random sample of 150000 rows into pandas...
Shape of your Data Set loaded: (150000, 13)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  4
    Number of Integer-Categorical Columns =  4
    Number of String-Categorical Columns =  4
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  1
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  0
    13 Predictors classified...
        No variables removed since no ID or low-information variables found in data se

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
periodo,int64,0.000000,0,201701.000000,201912.000000,No issue
customer_id,int64,0.000000,0,10001.000000,10635.000000,Column has 336 outliers greater than upper bound (10588.00) or lower than lower bound(9732.00). Cap them or remove them.
product_id,int64,0.000000,0,20001.000000,21286.000000,No issue
plan_precios_cuidados,int64,0.000000,0,0.000000,1.000000,No issue
cust_request_qty,int64,0.000000,0,1.000000,67.000000,Column has 16036 outliers greater than upper bound (3.50) or lower than lower bound(-0.50). Cap them or remove them.
cust_request_tn,float64,0.000000,NA,0.000100,551.561370,Column has 22317 outliers greater than upper bound (0.39) or lower than lower bound(-0.22). Cap them or remove them.
tn,float64,0.000000,NA,0.000100,547.878490,"Column has 22265 outliers greater than upper bound (0.39) or lower than lower bound(-0.22). Cap them or remove them., Column has a high correlation with ['cust_request_tn']. Consider dropping one of them."
cat1,object,0.245505,0,,,"368 missing values. Impute them with mean, median, mode, or a constant value such as 123., 1 rare categories: ['REF']. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"
cat2,object,0.245505,0,,,"368 missing values. Impute them with mean, median, mode, or a constant value such as 123., 4 rare categories: ['OTROS', 'ROPA MANCHAS', 'PROFESIONAL', 'TE']. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"
cat3,object,0.245505,0,,,"368 missing values. Impute them with mean, median, mode, or a constant value such as 123., 68 rare categories: Too many to list. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"


Number of All Scatter Plots = 10
All Plots done
Time to run AutoViz = 56 seconds 

 ###################### AUTO VISUALIZATION Completed ########################


In [ ]:


# Convertir la columna 'periodo' a datetime
#df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Crear características adicionales
df['mes'] = df['periodo'].dt.month
df['año'] = df['periodo'].dt.year
df['trimestre'] = df['periodo'].dt.quarter

# Ordenar los datos por cliente, producto y fecha
df = df.sort_values(by=['customer_id', 'product_id', 'año', 'mes'])

# Crear características lag para la cantidad requerida
df['cust_request_tn_lag1'] = df.groupby(['customer_id', 'product_id'])['cust_request_tn'].shift(1)
df['cust_request_tn_lag2'] = df.groupby(['customer_id', 'product_id'])['cust_request_tn'].shift(2)
df['cust_request_tn_lag3'] = df.groupby(['customer_id', 'product_id'])['cust_request_tn'].shift(3)

# Eliminar las filas con valores NaN generados por las operaciones lag
df = df.dropna()

# Verificar que las columnas lag se han creado correctamente
print(df[['cust_request_tn_lag1', 'cust_request_tn_lag2', 'cust_request_tn_lag3']].head())

# Definir las variables de entrada y la variable objetivo
features = ['customer_id', 'product_id', 'plan_precios_cuidados', 'mes', 'año', 'trimestre', 'cust_request_tn_lag1', 'cust_request_tn_lag2', 'cust_request_tn_lag3']
target = 'cust_request_tn'

X = df[features]
y = df[target]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Configuración del modelo
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Entrenamiento del modelo sin early stopping y sin verbose_eval
model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    num_boost_round=1000
)

# Evaluar el modelo
y_pred = model.predict(X_val, num_iteration=model.best_iteration)
mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {mae}')
